<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/Espell_check%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='E')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000943,E,38,383,분쇄하여,스크랩,스크랩분쇄가공처리
1,id_0001024,E,38,383,폐가전플라스틱케이스수거,사업장에서,"해체,분리 선별"
2,id_0001086,E,38,383,분쇄하여재생원료생산,플라스틱,재생원료
3,id_0003276,E,38,382,건설현장에서,고객의 의뢰를 받아,"폐기물, 골재대여"
4,id_0003533,E,38,382,고객의 요청으로,사업장에서,"텍스,석면철거"
...,...,...,...,...,...,...,...
2250,id_0998643,E,38,383,"폐,PVC,플라스틱","파쇄,분쇄",비금속 재생업
2251,id_0998908,E,38,383,폐PVC플라스틱폐기물,선별 파쇄,비금속원료물질
2252,id_0999035,E,38,383,"분쇄,파쇄 화학처리",폐비닐,폐합성수지 생산
2253,id_0999066,E,38,382,건설사요청에 의해,덤프차를 가지고,"건설,폐기물 수집운반"


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000943,E,38,383,분쇄하여 스크랩 스크랩분쇄가공처리
1,id_0001024,E,38,383,"폐가전플라스틱케이스수거 사업장에서 해체,분리 선별"
2,id_0001086,E,38,383,분쇄하여재생원료생산 플라스틱 재생원료
3,id_0003276,E,38,382,"건설현장에서 고객의 의뢰를 받아 폐기물, 골재대여"
4,id_0003533,E,38,382,"고객의 요청으로 사업장에서 텍스,석면철거"
...,...,...,...,...,...
2250,id_0998643,E,38,383,"폐,PVC,플라스틱 파쇄,분쇄 비금속 재생업"
2251,id_0998908,E,38,383,폐PVC플라스틱폐기물 선별 파쇄 비금속원료물질
2252,id_0999035,E,38,383,"분쇄,파쇄 화학처리 폐비닐 폐합성수지 생산"
2253,id_0999066,E,38,382,"건설사요청에 의해 덤프차를 가지고 건설,폐기물 수집운반"


In [8]:
df['digit_3'].value_counts()

381    704
383    568
370    436
382    371
360    147
390     29
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-kj9c4vl7
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-kj9c4vl7
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=f4b7cc8730b67298d7ee3bdfb229ba5546ad077f766933e6e4adc89781d4353f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5dfkv6va/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_E_spellcheck.csv', index=False, encoding='utf-8-sig')